---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [66]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [67]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
#swapped_states = {value:key for key, value in states.items()}

In [68]:
def get_list_of_university_towns():
#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State", "RegionName"]  )
    
#     The following cleaning needs to be done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
#     3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # open university_towns file and read data into a variable
    univ_file = open('university_towns.txt') 
    univ_text = univ_file.read()
    
    # split text string by new line \n
    univ = univ_text.split('\n')
    
    # capture the first state in the data to add to respective regions
    state = univ[0]
    
    # create empty list to store paired state and region names
    df_data = []
    
    # if a line in the univ data contains [edit], then replace the state value
    # Otherwise, create tupple with that state, region and append to new list
    for line in univ:
        if '[edit]' in line:
            state = line
            continue
        else:
            df_data.append([state, line])
    
    # convert list of tuples into a dataframe and name columns
    df = pd.DataFrame(df_data, columns=['State', 'RegionName'])
    
    # remove [edit] from state names
    df['State'] = df['State'].str.replace(r"\[edit\]", '')
    df['State'] = df['State'].str.strip()
    
    # replace blank RegionNames with NaN and then drop from df
    df['RegionName'] = df['RegionName'].replace('', np.nan)
    df.dropna(subset=['RegionName'], inplace=True)
    
    # split RegionNames column into two columns by (. Replace RegionName with everything before (.
    region_clean = df['RegionName'].str.split('(', n=1, expand=True)
    df['RegionName'] = region_clean[0]
    
    # remove any whitespace from RegionName
    df['RegionName'] = df['RegionName'].str.strip()
    
    return df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [69]:
# function to read and clean up GDP data
def GDP_data():
    # specify which rows to skip
    rows_skip = [1,2,3,4,5]

    # specify which columns to skip
    cols_skip = [0,1,2,3]

    # specify which columns to keep
    cols = [i for i in range(7) if i not in cols_skip]

    #usecols = cols
    # read Excel file, remove unnecessary columns and rows, and rename column headers
    gdp = (pd.read_excel('gdplev.xls', skiprows=rows_skip, usecols=cols,
                        names=['Year_Qtr', 'GDP in billions', 'GDP in billions chained 2009']))
    
    # drop any rows with NaN values. There are some blank rows after the header row
    gdp.dropna(axis=0, inplace=True)
    
    # grab the first four digits from the Year_Qtr column and convert to int
    gdp['year'] = gdp['Year_Qtr'].str[:4]
    gdp['year'] = gdp['year'].astype(int)
    
    # create df subset for 2000 and greater
    gdp_2000 = gdp[gdp['year'] >= 2000]
    gdp_2000.reset_index(drop=True, inplace=True)
    
    return gdp_2000


def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # call function to return cleaned up GDP data
    gdp = GDP_data()
    
    # calculate change in GDP from each year
    gdp['change_in_gdp'] = gdp['GDP in billions chained 2009'] - gdp['GDP in billions chained 2009'].shift(1)
    
    # formula to identify when current and previous years' change in GDP are negative and then next year change is positive
    beg_rec=(gdp['change_in_gdp'].shift(1)>0) & (gdp['change_in_gdp']<0) & (gdp['change_in_gdp'].shift(-1)<0)
    
    # create new column to identify the year & qtr when a recession starts
    gdp['start_rec'] = gdp['change_in_gdp'].where(beg_rec)
    
    # create new df where recession start is identified
    rec_start = gdp[gdp.start_rec.notnull()]
    
    return rec_start['Year_Qtr'].item()

get_recession_start()

'2008q3'

In [70]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # call function to return cleaned up GDP data
    gdp = GDP_data()
    
    # calculate change in GDP from each year
    gdp['change_in_gdp'] = gdp['GDP in billions chained 2009'] - gdp['GDP in billions chained 2009'].shift(1)
    
    # grab year & qtr of recession start and find its index
    start = get_recession_start() 
    start_ind = gdp.loc[gdp['Year_Qtr']==start].index[0]
    
    # limit dataset to start from recession start
    gdp_2008 = gdp.iloc[start_ind:]
    
    # formula to identify when current and previous years' change in GDP are positive
    end_rec=(gdp_2008['change_in_gdp'].shift(2)<0) & (gdp_2008['change_in_gdp'].shift(1)>0) & (gdp_2008['change_in_gdp']>0)
       
    # create new column to identify the year & qtr when a recession ends
    gdp_2008['end_rec'] = gdp_2008['change_in_gdp'].where(end_rec)
    
    # create new df where recession ends is identified
    rec_end = gdp_2008[gdp_2008.end_rec.notnull()]
    rec_end = rec_end['Year_Qtr'].head(1)
    
    return rec_end.item()

get_recession_end()

'2009q4'

In [71]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # call function to return cleaned up GDP data
    gdp = GDP_data()
    
    # calculate change in GDP from each year
    gdp['change_in_gdp'] = gdp['GDP in billions chained 2009'] - gdp['GDP in billions chained 2009'].shift(1)
    
    # grab year & qtr of recession start and end and find their indexes
    start = get_recession_start()
    end = get_recession_end()
    
    start_ind = gdp.loc[gdp['Year_Qtr']==start].index[0]
    end_ind = gdp.loc[gdp['Year_Qtr']==end].index[0]
    
    # limit dataset to from recession start to recession end
    rec = gdp.iloc[start_ind:end_ind]
    
    # find min of change_in_gdp to find recession bottom and return its index
    rec_bottom = rec.loc[rec['GDP in billions chained 2009'].idxmin()]
    
    # return Year & Qtr of recession bottom
    return rec_bottom['Year_Qtr']

get_recession_bottom()

'2009q2'

In [72]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    # read csv file and set index
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')#, index_col=['State', 'RegionName'])
    
    # use map function to rename state to state abbreviation
    housing['State'] = housing['State'].map(states)
    
    housing.set_index(['State', 'RegionName'], inplace=True)
    
    # find index of housing prices starting at 2000
    col_start = housing.columns.get_loc("2000-01")
    
    # create subset of housing data from 2000 on
    housing_2000 = housing.iloc[:,col_start:]
    
    # display Energy Supply as float vs scientific notation
    pd.options.display.float_format = '{:,}'.format
    
    # take the mean of the house prices for 3 columns at a time to calc mean housing prices per qtr
    mean_housing = pd.concat([housing_2000.ix[:,i:i+3].mean(axis=1) for i in range(0,len(housing_2000.columns),3)], axis=1)
    
    # create dict for column names
    col_rename = {}
    counter = 0
  
    for year in range(2000, 2017):
        for qtr in range(1, 5):
            item = {counter : (str(year) + "q" + str(qtr))}
            col_rename.update(item)
            counter += 1
    
    # rename df columns to columns in dict
    mean_housing.rename(columns=col_rename, inplace=True)
        
    return mean_housing
    
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,"515,466.6666666667","522,800.0","528,066.6666666666","532,266.6666666666","540,800.0","557,200.0","572,833.3333333334","582,866.6666666666","591,633.3333333334","587,200.0"
California,Los Angeles,"207,066.66666666666","214,466.66666666666","220,966.66666666666","226,166.66666666666","233,000.0","239,100.0","245,066.66666666666","253,033.33333333334","261,966.66666666666","272,700.0",...,"498,033.3333333333","509,066.6666666667","518,866.6666666667","528,800.0","538,166.6666666666","547,266.6666666666","557,733.3333333334","566,033.3333333334","577,466.6666666666","584,050.0"
Illinois,Chicago,"138,400.0","143,633.33333333334","147,866.66666666666","152,133.33333333334","156,933.33333333334","161,800.0","166,400.0","170,433.33333333334","175,500.0","177,566.66666666666",...,"192,633.33333333334","195,766.66666666666","201,266.66666666666","201,066.66666666666","206,033.33333333334","208,300.0","207,900.0","206,066.66666666666","208,200.0","212,000.0"
Pennsylvania,Philadelphia,"53,000.0","53,633.333333333336","54,133.333333333336","54,700.0","55,333.333333333336","55,533.333333333336","56,266.666666666664","57,533.333333333336","59,133.333333333336","60,733.333333333336",...,"113,733.33333333333","115,300.0","115,666.66666666667","116,200.0","117,966.66666666667","121,233.33333333333","122,200.0","123,433.33333333333","126,933.33333333333","128,700.0"
Arizona,Phoenix,"111,833.33333333333","114,366.66666666667","116,000.0","117,400.0","119,600.0","121,566.66666666667","122,700.0","124,300.0","126,533.33333333333","128,366.66666666667",...,"164,266.66666666666","165,366.66666666666","168,500.0","171,533.33333333334","174,166.66666666666","179,066.66666666666","183,833.33333333334","187,900.0","191,433.33333333334","195,200.0"
Nevada,Las Vegas,"132,600.0","134,366.66666666666","135,400.0","137,000.0","139,533.33333333334","141,733.33333333334","143,366.66666666666","146,133.33333333334","149,333.33333333334","150,933.33333333334",...,"170,066.66666666666","173,400.0","175,466.66666666666","177,500.0","181,600.0","186,766.66666666666","190,633.33333333334","194,600.0","197,200.0","199,950.0"
California,San Diego,"222,900.0","234,366.66666666666","245,433.33333333334","256,033.33333333334","267,200.0","276,266.6666666667","284,500.0","291,933.3333333333","301,233.3333333333","312,866.6666666667",...,"480,200.0","489,033.3333333333","496,433.3333333333","503,366.6666666667","512,066.6666666667","519,766.6666666667","525,466.6666666666","529,333.3333333334","536,233.3333333334","539,750.0"
Texas,Dallas,"84,466.66666666667","83,866.66666666667","84,866.66666666667","87,833.33333333333","89,733.33333333333","89,300.0","89,066.66666666667","90,900.0","92,566.66666666667","93,800.0",...,"106,633.33333333333","108,900.0","111,533.33333333333","113,700.0","121,133.33333333333","128,566.66666666667","134,600.0","140,500.0","144,600.0","149,300.0"
California,San Jose,"374,266.6666666667","406,566.6666666667","431,866.6666666667","455,500.0","470,666.6666666667","470,200.0","456,800.0","445,566.6666666667","441,433.3333333333","457,766.6666666667",...,"679,400.0","697,033.3333333334","714,933.3333333334","731,433.3333333334","756,733.3333333334","776,400.0","789,133.3333333334","803,600.0","818,933.3333333334","822,200.0"


In [103]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # grab mean housing data and reset index to be able to join to univ data
    housing = convert_housing_data_to_quarters()
    housing.reset_index(inplace=True)
    
    # grab recession start and bottom for price ratio calculation
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    
    # get univ town data and create new columns to indicate whether it is a univ town
    univ = get_list_of_university_towns()
    univ['is_univ_town'] = 1
    # set index to be able to join to housing data
    univ.set_index(['State', 'RegionName'],inplace=True)
    
    # join housing data with univ_town data
    df = housing.join(univ, on=['State', 'RegionName'], how='left')
    df.set_index(['State', 'RegionName'], inplace=True)
    
    # calculate the qtr before recession start
    rec_start_qtr = rec_start[-1:]
    qtr_before_rec = int(rec_start_qtr) - 1
    before_rec = rec_start[:-1] + str(qtr_before_rec)
    
    # calculate price ratio
    df['price_ratio'] = df[before_rec] / df[rec_bottom]
    
    # create separate df with univ towns only and drop NaN
    univ = df[df['is_univ_town'] == 1]
    univ.dropna(subset=['price_ratio'], inplace=True)
    
    # create separate df with non-univ towns and drop NaN
    not_univ = df[df['is_univ_town'] != 1]
    not_univ.dropna(subset=['price_ratio'], inplace=True)
    
    # run ttest on price_ratio comparing univ and non-univ towns
    ttest = ttest_ind(univ['price_ratio'], not_univ['price_ratio'])
    
    # compare p-value from ttest to 0.01. Since p-value is less than 0.01, we reject the null hypothesis
    # Null hypothesis: University towns do not have their mean housing prices less effected by recessions
    if ttest[1] < 0.01:
        different = True
    else:
        different = False
    
    # compare mean of price_ratio for univ and non-univ towns
    if univ['price_ratio'].mean() > not_univ['price_ratio'].mean():
        better = 'non-university town'
    else:
        better = 'university town'
        
    return (different, ttest[1], better)
    
run_ttest()

(True, 0.0027240637047614541, 'university town')